In [ ]:
from utils import *

In [ ]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="c643d8c8-4abb-47ba-808c-bf73f60161f1")

# Configure workspace
ws = Workspace.from_config(auth=interactive_auth)

In [ ]:
from azureml.core.model import InferenceConfig

env = get_env(name="sentiment-analysis", packages=["inference-schema", "tensorflow", "transformers"])

inference_config = InferenceConfig(
    environment=env,
    source_directory="code",
    entry_script="sentiment_analysis_batch.py",
)

In [ ]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [ ]:
from azureml.core import Model

service = Model.deploy(
    ws,
    "sentiment-analysis-batch",
    [],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

In [ ]:
print(service.get_logs())

In [ ]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}
data = {
    "Inputs": {
        "data": [
            "My ML skills are quite good.",
            "I didn't have good experience with ML."
        ]
    }
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.text)

In [ ]:
# service.delete()